# Figure 1
Assuming operation on google colab

In [1]:
# prepare environment
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# prepare environment
!pip install pycombat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# prepare environment
base_dir = "/content/drive/{YOUR PATH}"

import os
os.chdir(base_dir)

In [4]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

from src import difference as diff

In [5]:
# fxn for figure 1
def prep_data(batch,data,output,condition):
    """ prepare data for figure 1 """
    results = []
    negatives = []
    for c in condition:
        res,nega = diff.intra_dc(
            'madz',data,batch,key_compound='cmap_name',
            vehicle_control=c[1],control_names=['DMSO','medium'],drop_control=False,
            intra_batch=c[2],key_batch='batch_id',spearman=False
        )
        res.loc[:,'condition'] = [c[0]] * res.shape[0]
        res = res.reset_index()
        temp = nega.values.ravel()
        temp = pd.DataFrame({'intraDC':temp})
        temp.loc[:,'condition'] = [c[0]] * temp.shape[0]
        nega = temp.reset_index()
        results.append(res)
        negatives.append(nega)
    result = pd.concat(results,axis=0,join='inner')
    result = result.reset_index(drop=True)
    result.to_csv(output)
    negative = pd.concat(negatives,axis=0,join='inner')
    return result,negative,results,negatives


def plot_summary(result,negative):
    """ plot data for figure 1 """
    # plot the result
    print('--- main result ---')
    plt.rcParams['font.size'] = 14
    sns.displot(
        data=result, x='intraDC', hue='condition', kind='kde',
        palette='gist_stern', lw=2
        )
    plt.tight_layout()
    plt.show()

    # plot the null distribution
    print('--- null distribution ---')
    plt.rcParams['font.size'] = 14 
    sns.displot(
        data=negative, x='intraDC', hue='condition', kind='kde',
        palette='gist_stern', lw=2
        )
    plt.tight_layout()
    plt.show()


def plot_ks(condition,out_ks,result,negative,results,negatives):
    """ plot KS test """
    print('--- KS test ---')
    n_sample = len(condition)
    fig,axes = plt.subplots(n_sample,1,squeeze=False,tight_layout=True,figsize=(6,3*n_sample))
    plt.rcParams['font.size'] = 14
    ks_res = []
    imax = np.max(result['intraDC'])
    imin = np.min(result['intraDC'])
    nmax = np.max(negative['intraDC'])
    nmin = np.min(negative['intraDC'])
    vmax = np.max([imax,nmax])
    vmin = np.min([imin,nmin])
    count = 0
    idx = []
    for r,n,t in zip(results,negatives,condition):
        a = n['intraDC'].values.ravel()
        b = r['intraDC'].values.ravel()
        sns.distplot(a,ax=axes[count,0],hist=False,label='null',color='grey')
        sns.distplot(b,ax=axes[count,0],hist=False,label='intraDC',color='darkgoldenrod')
        axes[count,0].set_title(t[0])
        axes[count,0].spines['top'].set_visible(False)
        axes[count,0].spines['right'].set_visible(False)
        axes[count,0].set_xlim((vmin - np.abs(vmax) * 0.1,vmax + np.abs(vmax) * 0.1))
        ks_res.append(stats.ks_2samp(a,b))
        count += 1
        idx.append(t[0])
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()

    ks_res2 = pd.DataFrame(ks_res,index=idx)
    ks_res2.to_csv(out_ks)


def main(batch,data,output,out_ks,condition):
    """ runner for figure 1 """
    result,negative,results,negatives = prep_data(batch,data,output,condition)
    plot_summary(result, negative)
    plot_ks(condition, out_ks ,result, negative, results, negatives)
    print('>> completed')

In [6]:
# Figure 1
# use HT-HG-U133A_A dataset and compare the following conditions:
# - no batch correction (+), normalized by all samples in whole dataset
# - no batch correction (+), normalized by control samples in whole dataset
# - no batch correction (+), normalized by all samples in each batch
# - no batch correction (+), normalized by control samples in each batch

warnings.simplefilter('ignore')

# config
batch = pd.read_csv(base_dir + "/data/batch_HT-HG-U133A_selected.txt", sep='\t', index_col=0)
data = pd.read_pickle(base_dir + "/data/HTHGU133A_done_noBC_qn.pkl")
output = base_dir + "/output/Fig1/idc-summary_noBC.csv"
out_ks = base_dir + "/output/Fig1/ks_res_noBC.csv"
condition = [
    ('all in whole', False, False),
    ('control in whole', True, False),
    ('all in each', False, True),
    ('control in each', True, True)
    ]

# run
main(batch, data, output, out_ks, condition)

100%|██████████| 1074/1074 [00:04<00:00, 216.34it/s]


--- main result ---


ValueError: ignored

In [ ]:
# Figure 1
# use HT-HG-U133A_A dataset and compare the following conditions:
# - batch correction (+), normalized by all samples in whole dataset
# - batch correction (+), normalized by control samples in whole dataset
# - batch correction (+), normalized by all samples in each batch
# - batch correction (+), normalized by control samples in each batch

warnings.simplefilter('ignore')

# config
batch = pd.read_csv(base_dir + "/data/batch_HT-HG-U133A_selected.txt", sep='\t', index_col=0)
data = pd.read_pickle(base_dir + "/data/HTHGU133A_done_qn.pkl")
output = base_dir + "/output/Fig1/idc-summary.csv"
out_ks = base_dir + "/output/Fig1/ks_res.csv"

condition = [
    ('all in whole',False,False),
    ('control in whole',True,False),
    ('all in each',False,True),
    ('control in each',True,True)
    ]

# run
main(batch,data,output,out_ks,condition)